In [ ]:
import sympy
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.colors as mcolors
import dill
from tqdm import tqdm
from triangle import triangulate
from utils import (
    fill_stiffness_matrix_bell_preconditioned,
    orient_batch,
    get_precondition_terms,
)
from matplotlib import cm

dill.settings["recurse"] = True

### Get precomputed integrals

In [ ]:
biharmonic_functional = dill.load(
    open(
        "../calculations/bell_quintic_biharmonic_matrix_without_mixed_simplified", "rb"
    )
)
basis_integral_values = dill.load(
    open("../calculations/bell_quintic_biharmonic_matrix_right_values_simplified", "rb")
)

basis = dill.load(open("../calculations/bell_quintic_basis_transformed", "rb"))

### Initialize the domain

In [ ]:
t = np.linspace(0, 1, 50, endpoint=False)
points = np.stack((np.cos(2 * np.pi * t), np.sin(2 * np.pi * t))).T

seg = np.array(
    [[j, j + 1] for j in range(points.shape[0] - 1)] + [[points.shape[0] - 1, 0]]
)

### Run triangulation

In [ ]:
# S{n} - number of added steiner points

max_area = 0.009
min_angle = 30
order = 1

opts = f"pq{min_angle}a{max_area}o{order}en"

cndt = triangulate({"vertices": points, "segments": seg}, opts=opts)
points = cndt["vertices"]
triangles = cndt["triangles"]
edges = cndt["edges"]
neighbours = cndt["neighbors"]
vertex_marker_is_boundary = cndt["vertex_markers"]
edge_marker_is_boundary = cndt["edge_markers"]
print(
    f"Total number of points:\t\t{points.shape[0]}\nTotal number of elements:\t{triangles.shape[0]}"
)

### Visualize triangulation

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(7, 7))
ax.axis("equal")

cl = ["black", "brown"]

for eidx, edge in enumerate(edges):
    ax.plot(
        points[edge][:, 0],
        points[edge][:, 1],
        color=cl[edge_marker_is_boundary[eidx][0]],
    )

ax.scatter(*points.T, c=vertex_marker_is_boundary)

xmn = points[:, 0].min()
xmx = points[:, 0].max()
ymn = points[:, 1].min()
ymx = points[:, 1].max()


ax.set_xlim([xmn, xmx])
ax.set_ylim([ymn, ymx])

PLOT_TRIANGLE_NUMBER = False
PLOT_VERTEX_NUMBER = False

for index, triangle in enumerate(tqdm(points[triangles])):
    if PLOT_TRIANGLE_NUMBER is True:
        ax.text(triangle[:, 0].mean(), triangle[:, 1].mean(), f"{index}")

if PLOT_VERTEX_NUMBER is True:
    eps = 1e-2
    for p_index, point in enumerate(points):
        ax.text(point[0] + eps, point[1] + eps, f"{p_index}", fontsize=15)

fig.tight_layout()

plt.show()

### Initialize variables

In [ ]:
total_points = points.shape[0]
NUM = 6 * total_points

In [ ]:
x, y = sympy.symbols("x y")

### Define the right part of the equation

In [ ]:
F = sympy.Float(1)
F_x = F.diff(x)
F_y = F.diff(y)
F_xx = F_x.diff(x)
F_xy = F_x.diff(y)
F_yy = F_y.diff(y)

In [ ]:
right_part_values = np.zeros((total_points, 6))
right_part_values[:, 0] = sympy.lambdify((x, y), F)(*points.T)
right_part_values[:, 1] = sympy.lambdify((x, y), F_x)(*points.T)
right_part_values[:, 2] = sympy.lambdify((x, y), F_y)(*points.T)
right_part_values[:, 3] = sympy.lambdify((x, y), F_xx)(*points.T)
right_part_values[:, 4] = sympy.lambdify((x, y), F_xy)(*points.T)
right_part_values[:, 5] = sympy.lambdify((x, y), F_yy)(*points.T)

### Stiffnes matrix assembling

In [ ]:
triangles = orient_batch(triangles)
pts = points[triangles]

In [ ]:
x1 = right_part_values[triangles[:, 0]]
x2 = right_part_values[triangles[:, 1]]
x3 = right_part_values[triangles[:, 2]]

In [ ]:
cond = get_precondition_terms(points, triangles) 

In [ ]:
bilinear_form_ = biharmonic_functional(
    pts[:, 0, 0], pts[:, 0, 1], pts[:, 1, 0], pts[:, 1, 1], pts[:, 2, 0], pts[:, 2, 1]
).transpose(2, 0, 1)

right_part_ = basis_integral_values(
    pts[:, 0, 0],
    pts[:, 0, 1],
    pts[:, 1, 0],
    pts[:, 1, 1],
    pts[:, 2, 0],
    pts[:, 2, 1],
    *x1.T,
    *x2.T,
    *x3.T
)[:, 0].T

In [ ]:
bilinear_form_ = biharmonic_functional(
    pts[:, 0, 0], pts[:, 0, 1], pts[:, 1, 0], pts[:, 1, 1], pts[:, 2, 0], pts[:, 2, 1]
).transpose(2, 0, 1)

right_part_ = basis_integral_values(
    pts[:, 0, 0],
    pts[:, 0, 1],
    pts[:, 1, 0],
    pts[:, 1, 1],
    pts[:, 2, 0],
    pts[:, 2, 1],
    *x1.T,
    *x2.T,
    *x3.T
)[:, 0].T

matrix = np.zeros((NUM, NUM), dtype=np.float64)
b = np.zeros((NUM), dtype=np.float64)

for bilinear_form, right_part, element in tqdm(
    zip(bilinear_form_, right_part_, triangles), total=len(triangles)
):
    fill_stiffness_matrix_bell_preconditioned(
        matrix, b, bilinear_form, right_part, element, vertex_marker_is_boundary, cond,
        edges, points, edge_marker_is_boundary
    )

### Visualize stiffness matrix

In [ ]:
%matplotlib inline
fig, ax = plt.subplots(figsize=(6, 6))
ax.imshow(matrix != 0, cmap="gray")
plt.show()

### Solve the system

In [ ]:
c = np.linalg.solve(matrix.astype(np.float32), b.astype(np.float32))

### Get values

In [ ]:
_x_ = points[:, 0]
_y_ = points[:, 1]

U = (1 - x**2 - y**2) ** 2 / 64
U_x = sympy.lambdify((x, y), U.diff(x))(_x_, _y_)
U_y = sympy.lambdify((x, y), U.diff(y))(_x_, _y_)
U_xx = sympy.lambdify((x, y), U.diff(x, x))(_x_, _y_)
U_xy = sympy.lambdify((x, y), U.diff(x, y))(_x_, _y_)
U_yy = sympy.lambdify((x, y), U.diff(y, y))(_x_, _y_)
U = sympy.lambdify((x, y), U)(_x_, _y_)

F = c[0::6]
Fx = c[1::6] / cond
Fy = c[2::6] / cond
Fxx = c[3::6] / cond**2
Fxy = c[4::6] / cond**2
Fyy = c[5::6] / cond**2

In [ ]:
param = np.linspace(0, 1, 20)
tx, ty = np.meshgrid(param, param, indexing="ij")
mask = ty <= 1 - tx
tx = tx[mask]
ty = ty[mask]

In [ ]:
from utils import orient_batch

In [ ]:
X, Y, Z, Z_true = [], [], [], []
U = (1 - x**2 - y**2) ** 2 / 64

for element in triangles:
    x1, x2, x3 = points[element[0], 0], points[element[1], 0], points[element[2], 0]
    y1, y2, y3 = points[element[0], 1], points[element[1], 1], points[element[2], 1]

    ptx = x1 + tx * (x2 - x1) + ty * (x3 - x1)
    pty = y1 + tx * (y2 - y1) + ty * (y3 - y1)

    ptz = basis(
        x1,
        y1,
        x2,
        y2,
        x3,
        y3,
        
        F[element[0]],
        Fx[element[0]],
        Fy[element[0]],
        Fxx[element[0]],
        Fxy[element[0]],
        Fyy[element[0]],
           
        F[element[1]],
        Fx[element[1]],
        Fy[element[1]],
        Fxx[element[1]],
        Fxy[element[1]],
        Fyy[element[1]],
           
        F[element[2]],
        Fx[element[2]],
        Fy[element[2]],
        Fxx[element[2]],
        Fxy[element[2]],
        Fyy[element[2]],
           
        tx,
        ty,
    )

    X.append(ptx)
    Y.append(pty)
    Z.append(ptz)
    Z_true.append(sympy.lambdify((x, y), U)(ptx, pty))

X = np.concatenate(X)
Y = np.concatenate(Y)
Z = np.concatenate(Z)
Z_true = np.concatenate(Z_true)

## Surface plot

In [ ]:
from scipy.ndimage import map_coordinates
from scipy.interpolate import griddata
import plotly.graph_objs as go

x_values = np.linspace(-1.3, 1.3, 100)
y_values = np.linspace(-1.3, 1.3, 100)
XX, YY = np.meshgrid(x_values, y_values)
ZZ = griddata((X, Y), Z, (XX, YY), method='cubic', fill_value=0.0)

# fig = go.Figure(data=[go.Surface(z=ZZ, x=XX, y=YY)])

# # Customize the layout if needed
# fig.update_layout(
#     title='Surface Plot',
#     scene=dict(
#         xaxis_title='X Axis',
#         yaxis_title='Y Axis',
#         zaxis_title='Z Axis',
#     )
# )

# # Show the plot
# fig.show()

In [ ]:
plt.figure(figsize=(13, 13))
plt.contour(XX, YY, ZZ, levels=np.linspace(ZZ.min()*1.5, Z.max()*1.5, 40))

## Mesh plot

In [ ]:
import plotly.graph_objs as go

fig = go.Figure(data=[go.Mesh3d(x=X, y=Y, z=Z, intensity=Z)])


# Add layout options if needed
fig.update_layout(scene=dict(aspectmode="cube",),
                  width=900,
                  height=900)

# Show the plot
fig.show()